In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from scipy.optimize import differential_evolution

# 1. 数据加载与预处理
file_path = "database.xlsx"
sheet_name = "Maintable (M1) (CORRECTED)"
df = pd.read_excel(file_path, sheet_name=sheet_name)

# 假设最后一列为 failure_mode（预测目标），删除目标缺失的行
df = df.dropna(subset=[df.columns[-1]])

# 将最后一列作为目标，其余作为特征
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# 对特征进行缺失值处理：数值型用中位数，非数值型用众数
for col in X.columns:
    if pd.api.types.is_numeric_dtype(X[col]):
        X[col] = X[col].fillna(X[col].median())
    else:
        X[col] = X[col].fillna(X[col].mode()[0])

# 对非数值型特征进行 one-hot 编码
X = pd.get_dummies(X)

# 如果目标列是非数值型，则进行标签编码
if not pd.api.types.is_numeric_dtype(y):
    le = LabelEncoder()
    y = le.fit_transform(y)

# 输出数据基本情况
print("特征数据形状：", X.shape)
print("目标数据分布：\n", pd.Series(y).value_counts())

# 划分数据集：60%训练，20%验证，20%测试
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=1, stratify=y_temp)
# 现在：训练集 60%，验证集 20%，测试集 20%

# 2. 构造 5 层神经网络（这里的 5 层指 4 个隐藏层 + 输出层）
def create_model(random_state=None):
    # MLPClassifier 默认包含输入层（不算在 hidden_layer_sizes 内），本例中设定4个隐藏层
    # 隐藏层大小依次为：64, 32, 16, 8；激活函数采用 ReLU
    model = MLPClassifier(hidden_layer_sizes=(64, 32, 16, 8),
                          activation='relu',
                          solver='adam',
                          max_iter=200,
                          random_state=random_state)
    return model

# 3. 训练5个基模型
base_models = []
for i in range(5):
    model = create_model(random_state=i)
    model.fit(X_train, y_train)
    base_models.append(model)
    print(f"基模型 {i+1} 训练完成")

# 4. 获取验证集上各个模型的预测概率
# 对于多分类（或二分类），使用 predict_proba 返回每个类别的概率
val_preds = []
for i, model in enumerate(base_models):
    probas = model.predict_proba(X_val)  # shape: (n_samples, n_classes)
    val_preds.append(probas)
# 将 list 转换为 numpy 数组方便后续计算
# 注意：此时 val_preds 是 list，每个元素 shape 为 (n_val_samples, n_classes)

# 5. 定义目标函数：给定权重计算集成预测在验证集上的错误率（1 - accuracy）
def ensemble_error(weights):
    # 确保权重非负
    weights = np.clip(weights, 0, None)
    # 如果所有权重均为 0，则返回最大错误率
    if np.all(weights <= 0):
        return 1.0
    # 归一化权重，使得它们之和为 1
    w_norm = weights / np.sum(weights)
    # 计算加权平均概率
    # 初始化为全零矩阵，尺寸与单个模型的预测相同
    combined_proba = np.zeros(val_preds[0].shape)
    for w, preds in zip(w_norm, val_preds):
        combined_proba += w * preds
    # 根据概率预测类别（对于二分类和多分类均适用：选择概率最大的类别）
    combined_pred_labels = np.argmax(combined_proba, axis=1)
    # 计算准确率及错误率
    acc = np.mean(combined_pred_labels == y_val)
    error = 1 - acc
    return error

# 6. 使用 Differential Evolution 优化集成权重
# 每个基模型的权重取值范围设定为 [0, 1]
bounds = [(0, 1)] * len(base_models)
result = differential_evolution(ensemble_error, bounds, maxiter=50, popsize=15, tol=1e-6, seed=42)
optimal_weights = result.x
# 归一化最终权重
optimal_weights = np.clip(optimal_weights, 0, None)
optimal_weights = optimal_weights / np.sum(optimal_weights)
print("\n最优权重：", optimal_weights)

# 7. 在测试集上使用优化后的权重进行集成预测，并评估性能
test_preds = []
for model in base_models:
    probas = model.predict_proba(X_test)
    test_preds.append(probas)
# 计算加权平均概率
combined_proba_test = np.zeros(test_preds[0].shape)
for w, preds in zip(optimal_weights, test_preds):
    combined_proba_test += w * preds
# 预测类别：选择概率最大的类别
combined_pred_labels_test = np.argmax(combined_proba_test, axis=1)

# 计算评价指标
acc = accuracy_score(y_test, combined_pred_labels_test)
# 对于多分类问题，计算精确率和召回率时指定 average 参数
prec = precision_score(y_test, combined_pred_labels_test, average='weighted', zero_division=0)
rec = recall_score(y_test, combined_pred_labels_test, average='weighted', zero_division=0)
cm = confusion_matrix(y_test, combined_pred_labels_test)

print("\n测试集评估结果：")
print(f"准确率: {acc:.3f}")
print(f"加权精确率: {prec:.3f}")
print(f"加权召回率: {rec:.3f}")
print("混淆矩阵：")
print(cm)



特征数据形状： (491, 588)
目标数据分布：
 failure_mode
1.0    202
3.0    146
2.0    143
Name: count, dtype: int64
基模型 1 训练完成
基模型 2 训练完成
基模型 3 训练完成
基模型 4 训练完成
基模型 5 训练完成

最优权重： [0.08659315 0.03015454 0.22285125 0.24568879 0.41471227]

测试集评估结果：
准确率: 0.414
加权精确率: 0.334
加权召回率: 0.414
混淆矩阵：
[[ 0  0  0  0]
 [ 8 23 10  0]
 [ 0 11 18  0]
 [ 0  8 21  0]]
